In [38]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import CTransformers
from uuid import uuid4

In [39]:
load_dotenv()

True

In [40]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_NAME = "medical-chatbot"

In [41]:
# Step 1: Load PDF files using DirectoryLoader and PyPDFLoader
def load_pdf(directory_path):
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [42]:
extracted_data = load_pdf("../data/")

In [43]:
# Step 2: Split text into chunks using RecursiveCharacterTextSplitter
def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [44]:
text_chunks = split_text_into_chunks(extracted_data)
len(text_chunks)

3367

In [45]:
# Step 3: Download and generate Hugging Face model for embeddings
def generate_embeddings_model(model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [46]:
embeddings = generate_embeddings_model()

e:\programs\AI\practice\MedicalAssistant\mybot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [47]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [48]:
# Embed current loaded PDF and save embeddings in vector store
# uuids = [str(uuid4()) for _ in range(len(text_chunks))]
# vector_store.add_documents(documents=text_chunks, ids=uuids)

In [49]:
# Delete embeddings as i have alredy embedded doc
# vector_store.delete(ids=[uuids[1]])

In [50]:
# Test
results = vector_store.similarity_search(
    "What is the treatment for a common cold?",
    k=2
)
results

[Document(page_content='antibiotic several times a day to prevent infection. Ice-'),
 Document(page_content='serious if it progresses to pneumonia, therefore, antibi-otics may be required. In addition, however, there are amultitude of botanical and herbal medicines that can beformulated to treat bronchitis. Some examples includeinhaling eucalyptus or other essential oils in warm steam.Herbalists recommend a tea made of mullein ( Verbascum\nthapsus ), coltsfoot ( Tussilago farfara ), and anise seed\n(Pimpinella anisum ). Homeopathic medicine and tradi-\ntional Chinese medicine may also be very useful for')]

In [51]:
query = "What is the treatment for a common cold?"
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [52]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [53]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [54]:
retriever = vector_store.as_retriever(search_kwargs={'k': 2})
qa= RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever= retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [55]:
result=qa({"query": query})
print("Response : ", result["result"])

Response :  There are several treatments available for bronchitis, including antibiotics, botanical and herbal medicines, homeopathic medicine, and traditional Chinese medicine. However, it's important to note that antibiotics are usually only prescribed when the infection is severe or persistent, as they can have side effects and may not be effective against viral infections. If you have a common cold, there are several over-the-counter medications available that can help relieve symptoms such as congestion and cough. These include decongestants, expectorants, and cough suppressants. It's also important to stay hydrated by drinking plenty of fluids, getting enough rest, and avoiding close contact with others to prevent the spread of infection.
